In [88]:
%load_ext autoreload
%autoreload 2 

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, ExtraTreesClassifier, ExtraTreesRegressor
from IPython.display import display

from sklearn import metrics

In [90]:
PATH = 'data/'

In [91]:
df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False)

In [92]:
def display_all(df):
    with pd.option_context("display.max_rows",1000, 
                           "display.max_columns", 1000):
        display(df)

In [93]:
display_all(df_raw.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [94]:
display_all(df_raw.describe(include="all").T)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
PassengerId,891,NaN,NaN,NaN,446,257.354,1,223.5,446,668.5,891
Survived,891,NaN,NaN,NaN,0.383838,0.486592,0,0,0,1,1
Pclass,891,NaN,NaN,NaN,2.30864,0.836071,1,2,3,3,3
Name,891,891,"Nicola-Yarred, Miss. Jamila",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,891,2,male,577,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,714,NaN,NaN,NaN,29.6991,14.5265,0.42,20.125,28,38,80
SibSp,891,NaN,NaN,NaN,0.523008,1.10274,0,0,0,1,8
Parch,891,NaN,NaN,NaN,0.381594,0.806057,0,0,0,0,6
Ticket,891,681,CA. 2343,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fare,891,NaN,NaN,NaN,32.2042,49.6934,0,7.9104,14.4542,31,512.329


In [95]:
df_raw.to_feather('data/titanic_raw')

In [96]:
df_raw=pd.read_feather("data/titanic_raw")

In [97]:
## New function to add marrage column based on name prefix
strings = ['Mr.', 'Mrs.', 'MR', 'MRS']
def feature_extract(column, strings):
    features_present = [0] * len(column)
    for index, name in enumerate(column):
        for string in strings:
            if string in name:
                features_present[index] = 1
    return features_present

features = feature_extract(df_raw.Name, strings)

df_raw.insert(4, "Married", features, True)
    

In [98]:
train_cats(df_raw)

In [99]:
os.makedirs('tmp', exist_ok=True)

In [100]:
df_raw.to_feather("tmp/titanic_trained_raw")

In [101]:
df_raw=pd.read_feather('tmp/titanic_trained_raw')

In [102]:
df, y, nas = proc_df(df_raw, 'Survived')

In [103]:
def rmse(pred, actual): return math.sqrt(((pred-actual)**2).mean())

In [104]:
def print_score(m, df_train, y_train):
        result = [f'rmse train: {rmse(m.predict(df_train), y_train)}',
                 f'\nTotal set size: {len(y_train)}'
                 f'\nRsquared train: {m.score(df_train, y_train)}']
        print(result[0], result[1])
        if hasattr(m, 'oob_score_'):
            result.append(f'OOB: {m.oob_score_}')
            print(result[2])
            

In [105]:
m = RandomForestRegressor(n_jobs = -1, bootstrap=True, oob_score=True, n_estimators = 1000, max_depth=10, max_features=.5)
m.fit(df, y)
print_score(m, df, y)

rmse train: 0.18283876642000746 
Total set size: 891
Rsquared train: 0.8586507453955096
OOB: 0.49519050992853797


In [106]:
testset=pd.read_csv('data/test.csv')

In [107]:
features = feature_extract(testset.Name, strings)

testset.insert(3, "Married",features, True)

In [108]:
train_cats(testset)

In [109]:
df_test, _ , nas_test = proc_df(testset)

In [110]:
df_test.pop('Fare_na')

0      False
1      False
2      False
3      False
4      False
       ...  
413    False
414    False
415    False
416    False
417    False
Name: Fare_na, Length: 418, dtype: bool

In [111]:
df_test

,PassengerId,Pclass,Name,Married,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_na
0,892,3,207,1,2,34.5,0,0,153,7.8292,0,2,False
1,893,3,404,1,1,47.0,1,0,222,7.0000,0,3,False
2,894,2,270,1,2,62.0,0,0,74,9.6875,0,2,False
3,895,3,409,1,2,27.0,0,0,148,8.6625,0,3,False
4,896,3,179,1,1,22.0,1,1,139,12.2875,0,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,354,1,2,27.0,0,0,268,8.0500,0,3,True
414,1306,1,284,0,1,39.0,0,0,325,108.9000,23,1,False
415,1307,3,333,1,2,38.5,0,0,347,7.2500,0,3,False
416,1308,3,385,1,2,27.0,0,0,221,8.0500,0,3,True


In [112]:
df

,PassengerId,Pclass,Name,Married,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_na
0,1,3,109,1,2,22.0,1,0,524,7.2500,0,3,False
1,2,1,191,1,1,38.0,1,0,597,71.2833,82,1,False
2,3,3,354,0,1,26.0,0,0,670,7.9250,0,3,False
3,4,1,273,1,1,35.0,1,0,50,53.1000,56,3,False
4,5,3,16,1,2,35.0,0,0,473,8.0500,0,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,549,0,2,27.0,0,0,102,13.0000,0,3,False
887,888,1,304,0,1,19.0,0,0,15,30.0000,31,3,False
888,889,3,414,0,1,28.0,1,2,676,23.4500,0,3,True
889,890,1,82,1,2,26.0,0,0,9,30.0000,61,1,False


In [113]:
preds = m.predict(df_test)

In [114]:
preds_binary = preds.round()
preds_binary = [int(prediction) for prediction in preds_binary]

In [115]:
output=pd.DataFrame({'PassengerId' : df_test.PassengerId, "Survived": preds_binary})


In [116]:
output.to_csv('my_titanic_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
